## 分类问题的评价指标    

### 分类准确率的风险

假设现有一个癌症检测系统，输入体检信息，然后模块可以预测该用户是否患有癌症    

* 假设当前模型的分类（预测）准确度为99.9%  
* 如果癌症产生的概率只有0.01%     
* 因为癌症发生的概率很小，所以我们可以让系统对每一个输入都判定为无癌症，这样导致这样的一个模型的分类准确度为99.99%    
* 上面我们训练的一个机器学习模型的分类准确度为99.9%，该模型还没有我们直接将所有人都分类成无癌症的准确度高，这说明我们单单使用准确度来衡量一个分类算法是有问题的。    

**对于极度偏斜（Skewed Data）的数据，只使用分类准确度是远远不够的**   

###  引入混淆矩阵Confusion Matrix     

![confusion matrix](./img/confusionmatrix.png)

### 精准率（Precision）和召回率（Recall）      

* 精准率Precision    

    ![precision](./img/precision.png)       
    
    **精准率评估的是我们对一定数量的样本进行预测，然后预测成功的比例**       
    上图得到的结果是40%，表明如果我们对100个样本进行预测，预测正确的比例是40%   

* 召回率Recall     

    ![recall](./img/recall.png)    
    **召回率评估的是我们的模型可以从真实发生的一定事件中找出多少正确样本的比例**   x    
    上图计算得到的结果可以这样解释，在10000个样本中，一共有10个样本患有癌症，我们的模型可以正确分类出8个样本    
    也可以是如果有100个患病样本，我们的模型可以从中正确分类出80个        
    
* 二者的对比   

![contrast](./img/precisionandrecall.png)   

### demo    
上面说到对于极度偏斜（skewed）的数据，不能只用准确度来评价，下面给一个实例，分别用acc和precision，recall计算一下    

![demo](./img/demo.png)      

可以看到我们的模型在精准率和召回率的评估下效果非常差，几乎没有效果    

## 实现混淆矩阵，精准率和召回率    

In [1]:
import numpy as np
from sklearn import datasets

In [6]:
# 使用手写数字识别数据集  
digits = datasets.load_digits()
X = digits.data
# 要对target进行修改，所以这里copy一份
y = digits.target.copy()

为了体现精准率和召回率在某些方面对模型进行评估的优势，这里需要制造一种数据极偏的情况   

In [7]:
y[digits.target==9] = 1
y[digits.target!=9] = 0

In [8]:
# 使用逻辑回归模型对该数据集进行分类   
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/Users/barackbao/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [9]:
log_reg.score(X_test, y_test)

0.9755555555555555

In [10]:
# 下面来计算一下混淆矩阵   

# 首先需要使用模型进行一次预测
y_log_predict = log_reg.predict(X_test)

In [12]:
# 计算TN值
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 0))
TN(y_test, y_log_predict)

403

In [20]:
# 计算FP
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 1))
FP(y_test, y_log_predict)

2

In [21]:
# 计算FN
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 0))
FN(y_test, y_log_predict)

9

In [15]:
# 计算TP   
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 1))
TP(y_test, y_log_predict)

36

In [22]:
# 计算混淆矩阵
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])
confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [23]:
# 计算精准率和召回率  

def precision_score(y_true, y_predict):
    
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0
precision_score(y_test, y_log_predict)

0.9473684210526315

In [24]:
def recall_score(y_true, y_predict):
    
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0
recall_score(y_test, y_log_predict)

0.8

## 使用scikit-learn中的混淆矩阵，精准率和召回率  

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [26]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.9473684210526315

In [27]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.8